In [1]:
import sys
import numpy as np
import pandas as pd
sys.version

'3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:38:46) [MSC v.1929 64 bit (AMD64)]'

# Q 1. 서비스 이탈예측 데이터

In [89]:
"""
데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
x_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv
데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
"""

x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv", index_col='CustomerId')
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv", index_col='CustomerId')
x_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv", index_col='CustomerId')
x_label = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv", index_col='CustomerId')
x_train = x_train.drop('Surname', axis=1)
x_test = x_test.drop('Surname', axis=1)
def changeGender(x):
    return str(x).lstrip().capitalize()

x_train['Gender'] = x_train['Gender'].apply(changeGender)
x_test['Gender'] = x_test['Gender'].apply(changeGender)

In [92]:
print(x_train.shape)
display(x_train.info())
x_train.nunique()

(6499, 10)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6499 entries, 15799217 to 15706463
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      6499 non-null   int64  
 1   Geography        6499 non-null   object 
 2   Gender           6499 non-null   object 
 3   Age              6499 non-null   int64  
 4   Tenure           6499 non-null   int64  
 5   Balance          6499 non-null   float64
 6   NumOfProducts    6499 non-null   int64  
 7   HasCrCard        6499 non-null   int64  
 8   IsActiveMember   6499 non-null   int64  
 9   EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 558.5+ KB


None

CreditScore         459
Geography             3
Gender                2
Age                  69
Tenure               11
Balance            4162
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    6499
dtype: int64

In [26]:
display(x_train.head(3))
display(y_train.head(3))
x_train.shape

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
CustomerId,,,,,,,,,,
15799217,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
15748986,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
15722004,543,France,Female,31,4,138317.94,1,0,0,61843.73


,Exited
CustomerId,
15799217,0
15748986,0
15722004,0


(6499, 10)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, precision_score


x_train_dummies = pd.get_dummies(x_train)
y = y_train['Exited']

x_test_dummies = pd.get_dummies(x_test)

X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, train_size=0.7)

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [67]:
#model_score
predict_train_label = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

In [71]:
# 문제에서 묻는 것에 따라 모델 성능 확인하기
# 정확도 (accuracy) , f1_score , recall , precision -> model.predict로 결과뽑기
# auc , 확률이라는 표현있으면 model.predict_proba로 결과뽑고 첫번째 행의 값을 가져오기 model.predict_proba()[:,1]
print('train accuracy :', accuracy_score(Y_train, predict_train_label))
print('validation accuracy :', accuracy_score(Y_validation, predict_validation_label))
print()
print('train f1_score :', f1_score(Y_train, predict_train_label))
print('validation accuracy :', f1_score(Y_validation, predict_validation_label))
print()
print('train recall_score :', recall_score(Y_train, predict_train_label))
print('validation recall_score :', recall_score(Y_validation, predict_validation_label))
print()
print('train precision_score :', precision_score(Y_train, predict_train_label))
print('validation precision_score :', precision_score(Y_validation, predict_validation_label))
print()
print('train auc :', roc_auc_score(Y_train, predict_train_proba))
print('validation auc :', roc_auc_score(Y_validation, predict_validation_prob))

train accuracy : 1.0
validation accuracy : 0.861025641025641

train f1_score : 1.0
validation accuracy : 0.5758998435054773

train recall_score : 1.0
validation recall_score : 0.46464646464646464

train precision_score : 1.0
validation precision_score : 0.757201646090535

train auc : 1.0
validation auc : 0.8521086021086021


In [85]:
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:, 1]

print('test acc: ', accuracy_score(x_label, predict_test_label))
print('test f1_score: ', f1_score(x_label, predict_test_label))
print('test recall_score', recall_score(x_label, predict_test_label))
print('test precision_score: ', precision_score(x_label, predict_test_label))

test acc:  0.8628963153384748
test f1_score:  0.5698924731182796
test recall_score 0.44600280504908835
test precision_score:  0.7890818858560794


In [81]:
res = pd.DataFrame({'CustomerId': x_test.index, 'Exited': predict_test_label})
res
res.to_csv('./etc/datamanim_type2_q01.csv', index=False)

# Q 2. 이직여부 판단 데이터

In [ ]:
"""
데이터 설명 : 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists (참고, 데이터 수정)
"""

In [151]:
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv")

print(x_train.shape)
display(x_train.info())
display(x_train.head(2))
display(x_train.nunique())
display(y_train.head(2))

(12452, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.2+ MB


None

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17


enrollee_id               12452
city                        122
city_development_index       92
gender                        3
relevent_experience           2
enrolled_university           3
education_level               5
major_discipline              6
experience                   22
company_size                  8
company_type                  6
last_new_job                  6
training_hours              241
dtype: int64

,enrollee_id,target
0,25298,0.0
1,4241,1.0


In [152]:
x_train = x_train.drop(['enrollee_id', 'city', 'experience', 'company_size', 'company_type'], axis=1)
y_train = y_train.drop('enrollee_id', axis=1)
x_test = x_test.drop(['enrollee_id', 'city', 'experience', 'company_size', 'company_type'], axis=1)
y_test = y_test.drop('enrollee_id', axis=1)

In [147]:
(temp:=pd.get_dummies(x_train).corr().unstack()).loc[temp < 1].loc[temp > -0.99].map(abs).sort_values(ascending=False)[::2].head(20)

enrolled_university_Full time course         enrolled_university_no_enrollment              0.791319
education_level_Graduate                     education_level_Masters                        0.666694
education_level_High School                  major_discipline_STEM                          0.610270
education_level_Graduate                     education_level_High School                    0.429522
enrolled_university_no_enrollment            enrolled_university_Part time course           0.417877
gender_Male                                  gender_Female                                  0.393235
last_new_job_1                               last_new_job_>4                                0.387550
major_discipline_STEM                        education_level_Graduate                       0.374405
relevent_experience_No relevent experience   last_new_job_never                             0.373143
relevent_experience_Has relevent experience  last_new_job_never                            

In [153]:
x_train_dummies = pd.get_dummies(x_train)
y_train = y_train['target']
x_test_dummies = pd.get_dummies(x_test)
y_test = y_test['target']
print(x_train_dummies.shape)

(12452, 27)


In [157]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dummies, y_train, train_size=0.7)

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [165]:
val_predict = rf.predict(X_val)
val_predict_proba = rf.predict_proba(X_val)[:, 1]

print('val acc: ', accuracy_score(Y_val, val_predict))
print('val f1_score: ', f1_score(Y_val, val_predict))
print('val recall_score: ', recall_score(Y_val, val_predict))
print('val precision_score: ', precision_score(Y_val, val_predict))
print('val roc_auc_score: ', roc_auc_score(Y_val, val_predict_proba))

val acc:  0.7299250535331906
val f1_score:  0.38885523924894
val recall_score:  0.3436830835117773
val precision_score:  0.4476987447698745
val roc_auc_score:  0.6765502463061717


In [176]:
test_predict = rf.predict(x_test_dummies)
print('test acc: ', accuracy_score(y_test, test_predict))
x_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")
res = pd.DataFrame({'enrollee_id': x_test.enrollee_id, 'target': test_predict})
res.to_csv('./etc/datamanim_type2_q02.csv', index=False)

test acc:  0.7302415747092156


# Q 3. 정시 배송 여부 판단 (2회기출)

In [ ]:
"""
데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)
"""

In [178]:
x_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")
y_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv")

print(x_train_origin.shape)
display(x_train_origin.info())
display(x_train_origin.head(3))
display(x_train_origin.nunique())
display(y_train_origin.head(3))

(6598, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB


None

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674


ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3365
dtype: int64

,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1


In [182]:
x_train = x_train_origin.drop('ID', axis=1)
y_train = y_train_origin['Reached.on.Time_Y.N']
x_test = x_test_origin.drop('ID', axis=1)
y_test = y_test_origin['Reached.on.Time_Y.N']

In [184]:
x_train_dummies = pd.get_dummies(x_train)
x_test_dummies = pd.get_dummies(x_test)
x_train_dummies.shape

(6598, 24)

In [186]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dummies, y_train)

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [190]:
val_predict = rf.predict(X_val)
val_predict_proba = rf.predict_proba(X_val)[:, 1]

print('val acc: ', accuracy_score(Y_val, val_predict))
print('val f1_score: ', f1_score(Y_val, val_predict))
print('val recall_score: ', recall_score(Y_val, val_predict))
print('val precision_score: ', precision_score(Y_val, val_predict))
print('val roc_auc_score: ', roc_auc_score(Y_val, val_predict_proba))

val acc:  0.6690909090909091
val f1_score:  0.7055016181229774
val recall_score:  0.6592741935483871
val precision_score:  0.7587006960556845
val roc_auc_score:  0.7532823377291891


In [194]:
test_predict = rf.predict(x_test_dummies)
print('test acc: ', accuracy_score(y_test, test_predict))
res = pd.DataFrame({'ID': x_test_origin['ID'], 'Reached.on.Time_Y.N': test_predict})
res.to_csv('./etc/datamanim_type2_q03.csv', index=False)

test acc:  0.6593955919109293


# Q 4. 성인 건강검진 데이터

In [ ]:
"""
데이터 설명 : 2018년도 성인의 건강검 진데이터 (종속변수 : 흡연상태 1- 흡연, 0-비흡연 )
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_test.csv
데이터 출처 :https://www.data.go.kr/data/15007122/fileData.do (참고, 데이터 수정)
"""

In [14]:
x_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv")
y_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv")
x_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv")
y_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_test.csv")

print(x_train_origin.shape)
display(x_train_origin.info())
display(x_train_origin.head(3))
display(x_train_origin.nunique())
display(y_train_origin.head(3))

(44553, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           44553 non-null  int64  
 1   성별코드         44553 non-null  object 
 2   연령대코드(5세단위)  44553 non-null  int64  
 3   신장(5Cm단위)    44553 non-null  int64  
 4   체중(5Kg단위)    44553 non-null  int64  
 5   허리둘레         44553 non-null  float64
 6   시력(좌)        44553 non-null  float64
 7   시력(우)        44553 non-null  float64
 8   청력(좌)        44553 non-null  float64
 9   청력(우)        44553 non-null  float64
 10  수축기혈압        44553 non-null  float64
 11  이완기혈압        44553 non-null  float64
 12  식전혈당(공복혈당)   44553 non-null  float64
 13  총콜레스테롤       44553 non-null  float64
 14  트리글리세라이드     44553 non-null  float64
 15  HDL콜레스테롤     44553 non-null  float64
 16  LDL콜레스테롤     44553 non-null  float64
 17  혈색소          44553 non-null  float64
 18  요단백          44553 non-null  float

None

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,Y,0.0,Y
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,Y,0.0,Y
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,Y,0.0,N


ID             44553
성별코드               2
연령대코드(5세단위)       14
신장(5Cm단위)         13
체중(5Kg단위)         22
허리둘레             566
시력(좌)             19
시력(우)             17
청력(좌)              2
청력(우)              2
수축기혈압            130
이완기혈압             95
식전혈당(공복혈당)       276
총콜레스테롤           286
트리글리세라이드         390
HDL콜레스테롤         126
LDL콜레스테롤         289
혈색소              145
요단백                6
혈청크레아티닌           38
(혈청지오티)AST       219
(혈청지오티)ALT       245
감마지티피            488
구강검진수검여부           1
치아우식증유무            2
치석                 2
dtype: int64

,ID,흡연상태
0,0,0
1,1,0
2,2,1


In [15]:
print((x_train_origin['ID'] == y_train_origin['ID']).sum())

x_train = x_train_origin.drop(['ID', '구강검진수검여부'], axis=1)
y_train = y_train_origin['흡연상태']
x_test = x_test_origin.drop(['ID', '구강검진수검여부'], axis=1)
y_test = y_test_origin['흡연상태']

44553


In [16]:
def changeSex(x):
    if str(x) == 'F':
        return 0
    return 1
def changeTooth(x):
    if str(x) == 'N':
        return 0
    return 1
x_train['성별코드'] = x_train['성별코드'].apply(changeSex)
x_train['치석'] = x_train['치석'].apply(changeTooth)
x_test['성별코드'] = x_test['성별코드'].apply(changeSex)
x_test['치석'] = x_test['치석'].apply(changeTooth)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train)

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [21]:
val_predict = rf.predict(X_val)
val_predict_proba = rf.predict_proba(X_val)[:, 1]

print('Val acc: ', accuracy_score(Y_val, val_predict))
print('Val f1_score: ', f1_score(Y_val, val_predict))
print('Val recall_score: ', recall_score(Y_val, val_predict))
print('Val precision_score: ', precision_score(Y_val, val_predict))
print('Val roc_auc_score: ', roc_auc_score(Y_val, val_predict_proba))

Val acc:  0.7590447975581291
Val f1_score:  0.6782546152001918
Val recall_score:  0.693212447929429
Val precision_score:  0.6639286552452476
Val roc_auc_score:  0.8372522687885424


In [22]:
test_predict = rf.predict(x_test)
print('test acc: ', accuracy_score(y_test, test_predict))
res = pd.DataFrame({'ID': x_test_origin['ID'], '흡연상태': test_predict})
res.to_csv('./etc/datamanim_type2_q04.csv', index=False)

test acc:  0.9394918753927641


# Q 5. 자동차 보험가입 예측데이터

In [ ]:
"""
데이터 설명 : 자동차 보험 가입 예측 (종속변수 Response: 1 : 가입 , 0 :미가입)
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_test.csv
데이터 출처 :https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction(참고, 데이터 수정)
"""

In [23]:
x_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv")
y_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv")
x_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv")
y_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_test.csv")

print(x_train_origin.shape)
display(x_train_origin.info())
display(x_train_origin.head(3))
display(x_train_origin.nunique())
display(y_train_origin.head(3))

(304887, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304887 entries, 0 to 304886
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    304887 non-null  int64  
 1   Gender                304887 non-null  object 
 2   Age                   304887 non-null  int64  
 3   Driving_License       304887 non-null  int64  
 4   Region_Code           304887 non-null  float64
 5   Previously_Insured    304887 non-null  int64  
 6   Vehicle_Age           304887 non-null  object 
 7   Vehicle_Damage        304887 non-null  object 
 8   Annual_Premium        304887 non-null  float64
 9   Policy_Sales_Channel  304887 non-null  float64
 10  Vintage               304887 non-null  int64  
 11  id                    0 non-null       float64
dtypes: float64(4), int64(5), object(3)
memory usage: 27.9+ MB


None

,ID,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,id
0,0,Female,23,1,8.0,0,< 1 Year,Yes,61354.0,152.0,235,NaN
1,1,Male,27,1,28.0,1,< 1 Year,No,38036.0,152.0,207,NaN
2,2,Female,23,1,45.0,0,< 1 Year,Yes,25984.0,152.0,217,NaN


ID                      304887
Gender                       2
Age                         66
Driving_License              2
Region_Code                 53
Previously_Insured           2
Vehicle_Age                  3
Vehicle_Damage               2
Annual_Premium           46430
Policy_Sales_Channel       155
Vintage                    290
id                           0
dtype: int64

,ID,Response
0,0,0
1,1,0
2,2,1


In [25]:
x_train = x_train_origin.drop(['ID', 'id'], axis=1)
y_train = y_train_origin['Response']
x_test = x_test_origin.drop(['ID', 'id'], axis=1)
y_test = y_test_origin['Response']

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

x_train_dummies = pd.get_dummies(x_train)
x_test_dummies = pd.get_dummies(x_test)

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dummies, y_train)

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [30]:
val_predict = rf.predict(X_val)
val_predict_proba = rf.predict_proba(X_val)[:, 1]

print('val acc: ', accuracy_score(Y_val, val_predict))
print('val f1_score: ', f1_score(Y_val, val_predict))
print('val recall_score: ', recall_score(Y_val, val_predict))
print('val precision_score: ', precision_score(Y_val, val_predict))
print('val roc_auc_score: ', roc_auc_score(Y_val, val_predict_proba))

val acc:  0.8671249770407494
val f1_score:  0.1813773035887488
val recall_score:  0.12082705147533922
val precision_score:  0.36357744653272844
val roc_auc_score:  0.834476223326767


In [36]:
test_predict = rf.predict(x_test_dummies)
print('test acc: ', accuracy_score(y_test, test_predict))

test acc:  0.9666238093988612


# Q 6. 비행탑승 경험 만족도 데이터

In [ ]:
"""
데이터 설명 : 비행탑승 경험 만족도 (satisfaction 컬럼 : ‘neutral or dissatisfied’ or satisfied ) (83123, 24) shape
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_test.csv
데이터 출처 :https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction?select=train.csv (참고, 데이터 수정)
"""

# Q 7. 수질 음용성 여부 데이터

# Q 8. 약물 분류 데이터

# Q 9. 사기회사 분류 데이터

# Q 10. 센서데이터 동작유형 분류 데이터

# Q 11. 당뇨여부판단 데이터

----

# Q 12. 학생성적 예측 데이터

In [ ]:
"""
데이터 설명 : 학생성적 예측 (종속변수 :G3)
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/ishandutta/student-performance-data-set (참고, 데이터 수정)
"""

In [2]:
x_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv")
y_train_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv")
x_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv")
y_test_origin = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv")

print(x_train_origin.shape)
display(x_train_origin.info())
display(x_train_origin.head(3))
display(x_train_origin.nunique())
display(y_train_origin.head(3))
print(x_train_origin.columns)

(678, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   StudentID   678 non-null    int64 
 1   school      678 non-null    object
 2   sex         678 non-null    object
 3   age         678 non-null    int64 
 4   address     678 non-null    object
 5   famsize     678 non-null    object
 6   Pstatus     678 non-null    object
 7   Medu        678 non-null    int64 
 8   Fedu        678 non-null    int64 
 9   Mjob        678 non-null    object
 10  Fjob        678 non-null    object
 11  reason      678 non-null    object
 12  guardian    678 non-null    object
 13  traveltime  678 non-null    int64 
 14  studytime   678 non-null    int64 
 15  failures    678 non-null    int64 
 16  schoolsup   678 non-null    object
 17  famsup      678 non-null    object
 18  paid        678 non-null    object
 19  activities  678 non-null    object
 20  

None

,StudentID,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,1714,GP,F,18,U,GT3,T,4,3,other,...,no,4,3,3,1,1,3,0,14,13
1,1254,GP,F,17,U,GT3,T,4,3,health,...,yes,4,4,3,1,3,4,0,13,15
2,1639,GP,F,16,R,GT3,T,4,4,health,...,no,2,4,4,2,3,4,6,10,11


StudentID     678
school          2
sex             2
age             8
address         2
famsize         2
Pstatus         2
Medu            5
Fedu            5
Mjob            5
Fjob            5
reason          4
guardian        3
traveltime      4
studytime       4
failures        4
schoolsup       2
famsup          2
paid            2
activities      2
nursery         2
higher          2
internet        2
romantic        2
famrel          5
freetime        5
goout           5
Dalc            5
Walc            5
health          5
absences       31
G1             16
G2             16
dtype: int64

,StudentID,G3
0,1714,14
1,1254,15
2,1639,11


Index(['StudentID', 'school', 'sex', 'age', 'address', 'famsize', 'Pstatus',
       'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
       'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime',
       'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2'],
      dtype='object')


In [3]:
temp = x_train_origin.copy()
temp['y'] = y_train_origin['G3']

In [96]:
pd.get_dummies(temp[['Pstatus','y']]).corr()

,y,Pstatus_A,Pstatus_T
y,1.000000,-0.001743,0.001743
Pstatus_A,-0.001743,1.000000,-1.000000
Pstatus_T,0.001743,-1.000000,1.000000


In [16]:
x_train = x_train_origin.drop(['StudentID', 'sex', 'Pstatus', 'nursery', 'absences', 'activities', 'famsup', 'famrel','paid', 'Mjob', 'Fjob', 'schoolsup', 'guardian', 'famsize'], axis=1)
# x_train = x_train_origin.drop(['StudentID', 'sex', 'Pstatus'], axis=1)
# x_train = x_train_origin[['Medu', 'failures', 'G1', 'G2']]
y_train = y_train_origin['G3']
x_test = x_test_origin.drop(['StudentID', 'sex', 'Pstatus', 'nursery', 'absences', 'activities', 'famsup', 'famrel','paid', 'Mjob', 'Fjob', 'schoolsup', 'guardian', 'famsize'], axis=1)
# x_test = x_test_origin.drop(['StudentID', 'sex', 'Pstatus'], axis=1)
# x_test = x_test_origin[['Medu', 'failures', 'G1', 'G2']]
y_test = y_test_origin['G3']

x_train_dummies = pd.get_dummies(x_train)
x_test_dummies = pd.get_dummies(x_test)
print(x_train_dummies.shape)

(678, 27)


In [82]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train_dummies), columns=x_train_dummies.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test_dummies), columns=x_test_dummies.columns)
x_train_scaled.head(3)

,age,Medu,Fedu,traveltime,studytime,failures,freetime,goout,Dalc,Walc,...,reason_course,reason_home,reason_other,reason_reputation,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,0.428571,1.0,0.75,0.0,1.000000,0.0,0.50,0.50,0.00,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.285714,1.0,0.75,0.0,0.666667,0.0,0.75,0.50,0.00,0.5,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,0.142857,1.0,1.00,0.0,0.333333,0.0,0.75,0.75,0.25,0.5,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [109]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, ad

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dummies, y_train, random_state=42)

rf = RandomForestRegressor()
rf.fit(X_train, Y_train)

RandomForestRegressor()

In [81]:
val_predict = rf.predict(X_val)

print('val mse: ', mean_squared_error(Y_val, val_predict))
print('val mae: ', mean_absolute_error(Y_val, val_predict))
print('val r2: ', r2_score(Y_val, val_predict))
print('val mape: ', mean_absolute_percentage_error(Y_val, val_predict))
print('val rmse:', np.sqrt(mean_absolute_percentage_error(Y_val, val_predict)))

test_predict = rf.predict(x_test_dummies)

print('test mse: ', mean_squared_error(y_test, test_predict))
print('test r2: ', r2_score(y_test, test_predict))

val mse:  2.3690338379084968
val mae:  1.0244274509803921
val r2:  0.8297858237084955
val mape:  606131526319041.0
val rmse: 24619738.550988737
test mse:  3.0991868008841053
test r2:  0.7984592160123816


In [110]:
val_predict = rf.predict(X_val)

print('val mse: ', mean_squared_error(Y_val, val_predict))
print('val mae: ', mean_absolute_error(Y_val, val_predict))
print('val r2: ', r2_score(Y_val, val_predict))
print('val mape: ', mean_absolute_percentage_error(Y_val, val_predict))
print('val rmse:', np.sqrt(mean_absolute_percentage_error(Y_val, val_predict)))

test_predict = rf.predict(x_test_scaled)        # MinMaxScaler

print('test mse: ', mean_squared_error(y_test, test_predict))
print('test r2: ', r2_score(y_test, test_predict))

val mse:  2.3975451137254904
val mae:  1.045692156862745
val r2:  0.8277372994322472
val mape:  610105290696132.5
val rmse: 24700309.526322387
test mse:  118.30859972677594
test r2:  -6.693633676617939


In [71]:
val_predict = rf.predict(X_val)

print('val mse: ', mean_squared_error(Y_val, val_predict))
print('val mae: ', mean_absolute_error(Y_val, val_predict))
print('val r2: ', r2_score(Y_val, val_predict))
print('val mape: ', mean_absolute_percentage_error(Y_val, val_predict))
print('val rmse:', np.sqrt(mean_absolute_percentage_error(Y_val, val_predict)))

test_predict = rf.predict(x_test_scaled)        # StandardScaler

print('test mse: ', mean_squared_error(y_test, test_predict))
print('test r2: ', r2_score(y_test, test_predict))

val mse:  2.4331111718954253
val mae:  1.05228431372549
val r2:  0.8251818917388408
val mape:  577785340429120.8
val rmse: 24037165.815235388
test mse:  3.119926891247847
test r2:  0.7971104834769016


# Q 13. 중고차 가격 예측 데이터

# Q 14. 의료 비용 예측 데이터

# Q 15. 킹카운티 주거지 가격예측문제 데이터

# Q 16. 대학원 입학가능성 데이터

# Q 17. 레드 와인 퀄리티 예측 데이터

# Q 18. 현대 차량 가격 분류문제 데이터